For constructing quantum circuits we have to import QuantumCiruit from qiskit

In [ ]:
from qiskit import QuantumCircuit
qc=QuantumCircuit()

For including register in the circuit

In [ ]:
from qiskit import QuantumRegister
qr = QuantumRegister(2,'a')

Now we add these register in the circuit as follows,

In [ ]:
qc.add_register(qr)

In [ ]:
qc.qregs   ## qc.qregs is for displaying

In [ ]:
qc.draw()

Now we apply gates on the cicuit

In [ ]:
qc.h(qr[0])   ## applying Hadamard gate on the first qubit


In [ ]:
#qc.draw()

In [ ]:
#qc.x(qr[0]) ## applying Hadamard gate on the first qubit
#qc.draw()

In [ ]:
qc.cx(qr[1], qr[0])
qc.draw()

In [ ]:
from qiskit import Aer
sv_sim = Aer.get_backend('aer_simulator')    ##it is simulator for showing output

In [ ]:
for backend in Aer.backends():     
    print(backend)

In [ ]:
from qiskit import assemble
qc.save_statevector()         ## this converts output in to state vector
qobj = assemble(qc)           ## this assembles quantum circuit
job = sv_sim.run(qobj)         ## this is for running the job

In [ ]:
ket = job.result().get_statevector()
for amplitude in ket:
    print(amplitude)

In [ ]:
new_qc = QuantumCircuit(qr)

new_qc.initialize(ket, qr)

In [ ]:
new_qc.x(qr[0]) 

In [ ]:
from qiskit import assemble
new_qc.save_statevector()         ## this converts output in to state vector
qobj = assemble(new_qc)           ## this assembles quantum circuit
job = sv_sim.run(qobj) 
ket = job.result().get_statevector()
for amplitude in ket:
    print(amplitude)

Note that quantum register is for doing quantum operations and we have to use classical register for measuring the qubits.

In [ ]:
from qiskit import ClassicalRegister
cr = ClassicalRegister(2,'creg')

qc.add_register(cr)

In [ ]:
qc.measure(qr[0],cr[0])     ##measuring first quibit on first classical register
qc.measure(qr[1],cr[1])    ##measuring second quibit on second classical register

qc.draw()

In [ ]:
aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(qc, shots=100)      ##shots is the number of experiments
job = aer_sim.run(qobj, memory=True)                ##.run is for running the experiments

In [ ]:
hist = job.result().get_counts()
print(hist)

In [ ]:
from qiskit.visualization import plot_histogram

plot_histogram(hist)

In [ ]:
samples = job.result().get_memory()    #to get result of each experiment as a result
print(samples)

Note that the bits are labelled from right to left. So cr[0] is the one to the furthest right, and so on. As an example of this, here's an 8 qubit circuit with a Pauli  on only the qubit numbered 7, which has its output stored to the bit numbered 7.

In [ ]:
qubit = QuantumRegister(8)
bit = ClassicalRegister(8)
qc_2 = QuantumCircuit(qubit,bit)

qc_2.x(qubit[7])
qc_2.measure(qubit,bit) # this is a way to do all the qc.measure(qr8[j],cr8[j]) at once

qobj = assemble(qc_2, shots=8192)
aer_sim.run(qobj).result().get_counts()

If we need more than one quantum register and classical register in the circuit then we can do as follows,

In [ ]:
qc = QuantumCircuit(2,1)     ## where first arguments is corresponding to number of quantum regsiter and second
                             ##argument is corresponding to the number of classical register.

In [ ]:
qc.h(0)
qc.cx(0,1)
qc.measure(1,0)

qc.draw()

## Creating custom gates


Custom gates works as function or subroutine, we construct the subcircuit and we can use that part of the circuit
whenever we need it.

In [ ]:
sub_circuit = QuantumCircuit(3, name='toggle_cx')
sub_circuit.cx(0,1)
sub_circuit.cx(1,2)
sub_circuit.cx(0,1)
sub_circuit.cx(1,2)

sub_circuit.draw()

In [ ]:
toggle_cx = sub_circuit.to_instruction()   ##we convert this circuit to the custom gate

In [ ]:
qr = QuantumRegister(4)
new_qc = QuantumCircuit(qr)

new_qc.append(toggle_cx, [qr[1],qr[2],qr[3] ])

new_qc.draw()

In [11]:
from qiskit import QuantumCircuit
import numpy as np

def deutsch_problem():
    """Returns a circuit that carries out the function
    from Deutsch's problem.
    Args:
        seed (int): If set, then returned circuit will
            always be the same for the same seed.
    Returns: QuantumCircuit
    """
    problem = QuantumCircuit(2)
    if np.random.randint(2):
        print("Function is balanced.")
        problem.cx(0, 1)
    else:
        print("Function is constant.")
    if np.random.randint(2):
        problem.x(1)
    return problem

In [25]:
def deutsch(function):
    """Implements Deutsch's algorithm.

    Args:
        function (QuantumCircuit): Deutsch function to solve.
            Must be a 2-qubit circuit, and either balanced,
            or constant.
    Returns:
        bool: True if the circuit is balanced, otherwise False.
    """

    # your code here
    f=True
    qc=QuantumCircuit(2)
    qc.x(1)
    #qc.draw()
    return (f)
problem=deutsch_problem()
#problem.draw()


Function is balanced.


In [34]:
from qiskit import ClassicalRegister
from qiskit import QuantumRegister
qr = QuantumRegister(2)
qc=QuantumCircuit(qr)

qc.x(1)
qc.h(0)
qc.h(1)
from qiskit import Aer
sv_sim = Aer.get_backend('aer_simulator')
from qiskit import assemble
qc.save_statevector()
qobj = assemble(qc)
job = sv_sim.run(qobj)
ket = job.result().get_statevector()
problem.initialize(ket)
#qc.append(problem, [qr[0],qr[1] ])
problem.h(0)
cr = ClassicalRegister(1)

problem.add_register(cr)
sv_sim = Aer.get_backend('aer_simulator')
problem.save_statevector()
qobj = assemble(problem)
job = sv_sim.run(qobj)
ket1 = job.result().get_statevector()
qc1=QuantumCircuit(qr)
qc1.initialize(ket1)
#problem.measure(qr[0],cr[0])
problem.draw()


Simulation failed and returned the following error message:
ERROR: Failed to load qobj: Duplicate key "statevector" in save instruction.


QiskitError: 'You have to select a circuit or schedule when there is more than one available'

In [8]:
from qiskit import Aer
from qiskit import assemble
aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(qc, shots=100)      ##shots is the number of experiments
job = aer_sim.run(qobj, memory=True)  
hist = job.result().get_counts()
print(hist)

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Circuit circuit-86 contains invalid instructions {"gates": {circuit-85}} for "statevector" method.


QiskitError: 'ERROR:  [Experiment 0] Circuit circuit-86 contains invalid instructions {"gates": {circuit-85}} for "statevector" method. ,  ERROR: Circuit circuit-86 contains invalid instructions {"gates": {circuit-85}} for "statevector" method.'